
# Trabalho 2 - Tópicos em IA 
## (**Parte 2**)

### Realizando Predições e Visualizando Métricas
---


## Introdução

Neste ponto, os modelos foram treinados e estão salvos em `/documents/bertimbau-(baseline | alternativo)-cls-ptbr`

Vamos carregá-los, realizar as predições e visualizar métricas

**Observação**: Para realizar as predições, precisamos do `test_dataset`, que criamos na primeira parte do trabalho.

Para isso, podemos replicar o código aqui, ou salvar e pegar apenas ele. Salvei na pasta documents, que contém o corpus de notícias e os modelos. Para replicar este notebook, os passos do notebook anterior devem ser replicados desde o começo, e o corpus de notícia deve estar corretamente rastreado (ou, quem sabe, no futuro, pegar o modelo direto do meu hugging face).


---

## Configs Iniciais

#### configurações iniciais do path

In [1]:

import os, sys
root = os.path.abspath('../')
src_path = os.path.join(root, 'src')
sys.path.append(root)

#### imports

In [2]:
from safetensors.torch import load_file
from torch import load as t_load
import torch

from src.custom_bertimbau_classifier import CustomBertimbauClassifier
from src.baseline_bertimbau_classifier import BaselineBertimbauClassifier
from src.dataset_loader import DatasetLoader, FakeNewsDataset
import src.config as cfg

import numpy as np
import random
import evaluate

from transformers import (
    BertForSequenceClassification,
    Trainer, TrainingArguments,
    PreTrainedTokenizerFast
    )

c:\Users\mathe\OneDrive\Área de Trabalho\fake-news-detector\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### algumas funções simples para simplificar o script

In [3]:

get_checkpoint_dir = lambda base_alt: os.path.join(root,"documents",f"bertimbau-{base_alt}-cls-ptbr","checkpoint-630") 
get_state_dict = lambda base_alt: load_file(os.path.join(get_checkpoint_dir(base_alt),"model.safetensors")) 

# print(type(get_checkpoint_dir("baseline")))

#### tornando as libs determinísticas

In [4]:
#physics are relative, but this code gonna be DETERMINISTIC BABY! (sim, copiei do main.ipynb)
random.seed(cfg.RANDOM_SEED) 
np.random.seed(cfg.RANDOM_SEED) 
torch.manual_seed(cfg.RANDOM_SEED)

torch.backends.cudnn.deterministic = True

## Carregando Trainer e Dataset para Predição

### Compute Metrics

Devemos trazer de volta a função customizada de computar métricas

In [5]:
acc_metric = evaluate.load("accuracy")
prec_metric = evaluate.load("precision")
rec_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    
    return {
        "accuracy": acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "precision": prec_metric.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall": rec_metric.compute(predictions=preds, references=labels, average="macro")["recall"],
        "f1": f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"]
    }


### Load do Dataset para Testes

#### load de `test_dataset`
Reproduzindo o código que processa o dataset de notícias...

Este código é interessante pois traz todos os datasets, o tokenizer e os kwargs do `BertForSequenceClassification` são facilmente recuperados.

Porém, existe uma forma alternativa de capturar o dataset e o tokenizer, através dos arquivos `tokenizer.json` e, caso criado, `test_dataset.pkl`

Para o dataset, criei das duas formas, para fins demosntrativos. O mesmo quanto ao tokenizer 

In [ ]:
path = os.path.join(root, cfg.PATH_TO_DATASET)
ds_loader = DatasetLoader(
    path=path,
    model_name=cfg.BERTIMBAU,
    max_len=cfg.SEQ_LEN
)

#conjunto para teste, validação e treino
# _, _, test_dataset = (ds_loader
#                         .load_dataset(seed=cfg.RANDOM_SEED)
#                         .get_datasets())

# len(test_dataset)


1080

#### load de `test_dataset` (_método diferenciado_)

Alternativamente, é possível poupar retrabalho. Tendo setado `salvar_test_dataset = True` no último bloco de código da seção "Carregando o Dataset de Notícias", no fluxo do notebook principal, temos o arquivo `test_dataset.pkl` salvo em documents. Para extraí-lo, basta utilizar a biblioteca `pickle`



In [ ]:
#é necessário ter o tipo do objeto carregado pelo pickle presente no código.
import pickle
test_dataset_path = os.path.join(root, "documents", "test_dataset.pkl")

with open(test_dataset_path, 'rb') as ds:
    test_dataset = pickle.load(ds)


print(type(test_dataset))

<class 'src.dataset_loader.FakeNewsDataset'>


### Load do Tokenizer

Aqui mostro as duas formas de capturar o tokenizer.

#### forma com o `ds_loader` carregado

(certifique-se de ter carregado o Dataset Loader para executar)

In [ ]:
# tokenizer = ds_loader.get_tokenizer()

#### _método diferenciado_: 
forma com `tokenizer.json` salvo devido a `save_strategy` em `TrainingArguments`

In [ ]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file=os.path.join(get_checkpoint_dir("baseline"),"tokenizer.json"))

### Load do Trainer
Para isso, aproveitaremos dos arquivos salvos devido à configuração `save_strategy`, de `TrainingArguments`

#### Trainer do Baseline

In [23]:

baseline_checkpoint_dir =  get_checkpoint_dir("baseline")
model_kwargs = ds_loader.get_labels_mapping() #TODO: descobrir se é necessário fora do treinamento

baseline_model = BertForSequenceClassification.from_pretrained(
    baseline_checkpoint_dir,
    **model_kwargs
).to(cfg.DEVICE)

# baseline_training_args = TrainingArguments.from_pretrained(baseline_checkpoint_dir)
baseline_training_args = t_load(os.path.join(baseline_checkpoint_dir, "training_args.bin"), weights_only=False)
assert isinstance(baseline_training_args, TrainingArguments)

baseline_training_args.eval_strategy = "no"

baseline_trainer = Trainer(
    model=baseline_model,
    args=baseline_training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\mathe\AppData\Local\Temp\ipykernel_21864\3397625602.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  baseline_trainer = Trainer(


In [11]:
print(baseline_checkpoint_dir)

c:\Users\mathe\OneDrive\Área de Trabalho\fake-news-detector\documents\bertimbau-baseline-cls-ptbr\checkpoint-630


#### Trainer do modelo alternativo

(lembrando, o tokenizer é o mesmo para as duas versões do modelo)

In [ ]:
alt_checkpoint = get_checkpoint_dir("alternativo")
alt_training_args = t_load(os.path.join(alt_checkpoint, "training_args.bin"), weights_only=False)

assert tokenizer is not None

alt_model = CustomBertimbauClassifier() #carrega lá do hugging face, denovo

alt_state_dict = load_file(os.path.join(alt_checkpoint, "model.safetensors"))
alt_model.load_state_dict(alt_state_dict)

alt_training_args.eval_strategy = "no"
print(alt_training_args.output_dir)

alt_trainer = Trainer(
    alt_model,
    alt_training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



print("caminho pro checkpoint:", alt_checkpoint)
alt_trainer

c:\Users\mathe\OneDrive\Área de Trabalho\fake-news-detector\documents\bertimbau-alternativo-cls-ptbr
caminho pro checkpoint: c:\Users\mathe\OneDrive\Área de Trabalho\fake-news-detector\documents\bertimbau-alternativo-cls-ptbr\checkpoint-630


C:\Users\mathe\AppData\Local\Temp\ipykernel_21864\15985691.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  alt_trainer = Trainer(


## Realizando Predições

In [24]:
baseline_predictions = baseline_trainer.predict(test_dataset)

In [14]:
baseline_predictions.metrics

{'test_loss': 7.033188819885254,
 'test_model_preparation_time': 0.0319,
 'test_accuracy': 0.018518518518518517,
 'test_precision': 0.01849209833187006,
 'test_recall': 0.018518518518518517,
 'test_f1': 0.018505054939025226,
 'test_runtime': 142.8735,
 'test_samples_per_second': 7.559,
 'test_steps_per_second': 0.476}

In [15]:
alternative_predictions = alt_trainer.predict(test_dataset=test_dataset)

c:\Users\mathe\OneDrive\Área de Trabalho\fake-news-detector\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [16]:
alternative_predictions.metrics

{'test_loss': 8.360626220703125,
 'test_model_preparation_time': 0.0153,
 'test_accuracy': 0.018518518518518517,
 'test_precision': 0.018511913743672753,
 'test_recall': 0.018518518518518517,
 'test_f1': 0.018515152658273863,
 'test_runtime': 161.8485,
 'test_samples_per_second': 6.673,
 'test_steps_per_second': 0.42}

In [17]:

x = alt_model.load_state_dict(alt_state_dict, strict=False)
print(x)

<All keys matched successfully>
